# 1. 详解神经网络的组成部分

## 1.1 层 —— 神经网络的基本组成

In [7]:
import time

from torch.nn import Linear
import torch

DEFAULT_DEVICE = 'mps'

layer = Linear(in_features=10, out_features=5, bias=True, device=DEFAULT_DEVICE)

layer(torch.randn(1, 10, device=DEFAULT_DEVICE))

layer.weight, layer.bias

(Parameter containing:
 tensor([[-0.1093, -0.2341,  0.1806,  0.2400, -0.2131, -0.2347,  0.2537, -0.0582,
          -0.0757, -0.0850],
         [-0.1884,  0.2147,  0.2702, -0.0637,  0.2283,  0.1969, -0.1904,  0.1933,
           0.1495,  0.2835],
         [-0.1350,  0.0072, -0.1603,  0.0334, -0.0517,  0.1345, -0.0337, -0.3087,
           0.2901,  0.1656],
         [ 0.1759, -0.2352,  0.2515, -0.1472,  0.1112, -0.3137, -0.2840,  0.0612,
          -0.1526,  0.2599],
         [ 0.1245,  0.2647,  0.1110, -0.1463, -0.1335, -0.0825,  0.0082, -0.1154,
          -0.2485,  0.1287]], device='mps:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.1394,  0.1253,  0.0173, -0.0545, -0.1697], device='mps:0',
        requires_grad=True))

一些流行的非线性函数：
* sigmoid
* tanh
* ReLU
* Leaky ReLU

## 1.2 PyTorch 中的非线性激活函数

In [8]:
from torch.nn import ReLU, MSELoss

simple_data = torch.Tensor([1, 2, -1, -1])

relu = ReLU()
relu(simple_data)

tensor([1., 2., 0., 0.])

In [9]:
mse_loss = MSELoss()

X = torch.randn(3, 5, requires_grad=True)
y = torch.randn(3, 5)

output = mse_loss(X, y)

output, output.backward()

(tensor(4.1083, grad_fn=<MseLossBackward0>), None)

PyTorch 提供的优化器
* ADADELTA
* Adagrad
* Adam
* SparseAdam
* Adamax
* ASGD
* LBFGS
* RMSProp
* Rprop
* SGD

## 1.3 使用深度学习对图像进行分类

处理数据集，处理成 ImageFolder 可以处理的路径格式。

In [10]:
from dot_cat_dataset import DOG_CAT_DATASET_PATH
from glob import glob
import os.path
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

# 数据转换，图片转换为张量并正则化
simple_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


def is_valid_file(file: str):
    return not file.split('/')[-1].startswith("._") and file.endswith('jpg')


full_images = ImageFolder(DOG_CAT_DATASET_PATH, transform=simple_transform, is_valid_file=is_valid_file)

full_images.class_to_idx, full_images.classes

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/WTTCH/数据集/archive/PetImages'

将张量可视化

In [ ]:
import matplotlib.pyplot as plt


def imshow(inp: torch.Tensor):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

imshow(full_images[0][0])

切分数据集

In [ ]:
from torch.utils.data import random_split
from torch.utils.data.dataset import Subset
from torch.utils.data import DataLoader

# train_dataset, test_dataset = random_split(full_images, [23000, len(full_images) - 23000]) # type: Subset, Subset
train_dataset, test_dataset = random_split(full_images, [1000, 200]) # type: Subset, Subset

train_data_gen = DataLoader(train_dataset, batch_size=64, num_workers=3, shuffle=True)
test_data_gen = DataLoader(test_dataset, batch_size=64, num_workers=3, shuffle=True)

使用 ResNet 神经网络

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

model_ft = resnet18(weights=[ResNet18_Weights.DEFAULT], progress=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = torch.nn.Linear(num_ftrs, 2)

model_ft.to(device='mps')

训练模型:

In [ ]:
from torch.nn.modules.loss import _Loss
import torch.nn as nn
import torch.optim as optim



def train_model(model: nn.Module, loss_fn: _Loss, optimizer: optim.Optimizer, dataset: DataLoader):
    size = len(dataset.dataset.dataset)
    
    model.train()
    
    for batch, (X, y) in enumerate(dataset):
        X = X.to(device='mps')
        y = y.to(device='mps')
        # 计算
        pred = model(X)
        # 损失
        loss = loss_fn(pred, y)
        
        # 反向传播
        loss.backward()
        # 收集梯度
        optimizer.step()
        # 梯度归零
        optimizer.zero_grad()
        
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        


In [ ]:

# 损失函数和优化器
learning_rate = 0.001
cross_entropy_loss_fn = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)


train_model(model_ft, cross_entropy_loss_fn, optimizer_ft, train_data_gen)